In [18]:
# Install dependencies
# Only need to run once
!pip3 install ikpy -U
!pip3 install pyserial

In [1]:
from enum import Enum

import serial
import numpy as np
import ikpy
import ikpy.chain
import time

# Servo class to store the servo data
class Servo(object):
    # Constructor
    # ID: Servo ID
    # angle: Angle of the servo in degrees
    # raw: Raw value of the servo in CCR
    # minCCR: Minimum CCR value of the servo (default: 1000 on MG-Series Servos)
    # maxCCR: Maximum CCR value of the servo (default: 4999 on MG-Series Servos)
    def __init__(self, ID, angle, raw, minCCR=1000, maxCCR=4999):
        # Set initial values
        self.id = ID
        self.angle = angle
        self.raw = raw
        self.minCCR = minCCR
        self.maxCCR = maxCCR
        
    # Set the angle of the servo
    # angle: Angle of the servo in degrees
    def setAngle(self, angle):
        self.angle = angle
        # Calculate the raw value of the servo at the given angle
        # Use linear interpolation to calculate the raw value
        self.raw = int(self.angle * (self.maxCCR - self.minCCR) / 180 + self.minCCR)
        # Because data sent as string, convert the raw value to string
        return str(self.raw)

# Stance enum to store the stance data
# Don't really need, can be removed in the future
class Stance(Enum):
    SBY = np.array([200, 120]) # Stand By
    SCN = np.array([180, 90]) # Scanning

    # Get the value of the stance
    def __getitem__(self, item):
        return self.value[item]

# ArmRobot class to control the robot
class ArmRobot(object):
    # Constructor
    # distTh: Distance threshold for the robot to detect an object (mm)
    # liftTh: Lift threshold for the robot to lift an object (mm)
    # moveTgt: Target rotation for the robot to move an object (degree)
    # scnArea: Scanning area for the robot to scan an object (degree)
    # com: COM port of the robot
    # urdfPath: URDF file path of the robot
    def __init__(self, distTh = 100, liftTh=200, moveTgt=180, scnArea = 160, com='COM10', urdfPath="Arb_old.urdf"):
        # Set the initial values
        self.distTh = distTh
        self.liftTh = liftTh
        self.moveTgt = moveTgt
        self.scnArea = scnArea
        self.stance = np.array([0,0])
        # Data format the robot stream
        self.data = {'DIST': 0, 'ROT0': 0, 'ROT1': 0, 'ROT2': 0, 'ROT3': 0, 'ROT4': 0, 'RAW0': 1000, 'RAW1': 1000, 'RAW2': 1000,'RAW3': 1000, 'RAW4': 1000}
        # Easing function for the servo movement
        self.ease = (lambda x: x**2 / (x**2 + (1-x)**2))(np.linspace(0, 1, 256))
        
        # Open the serial port
        self.serial = serial.Serial(com, 115200, timeout=1)
        # Create the servo objects
        self.servos = np.array([Servo(i, int(self.data[f"ROT{i}"]), int(self.data[f"RAW{i}"])) for i in range(5)])
        # Load the URDF file of the robot
        self.model = ikpy.chain.Chain.from_urdf_file(urdfPath)
        # Set the initial stance of the robot
        self.setStance(90, Stance.SBY[0], Stance.SBY[1], 0)

    # Read and parse the data from the robot
    # ret: Data to return
    def readAndParse(self, ret = None):
        # Send the command to request the data from the robot
        self.serial.write("L 0 0".encode())
        data = self.serial.readline()
        # Wait until the data is received
        while not data:
            data = self.serial.readline()
            
        # Parse the data
        data = data.decode().split(" ")
        self.data = {sd[0]: int(sd[1]) for sd in np.array([d.split(":") for d in data])}
        # Return the requested data
        if ret is not None:
            for req in ret.split(" "):
                if req not in self.data:
                    return None
            return np.array([self.data[req] for req in ret.split(" ")])

    # Set the joints of the robot
    # targetAngles: Target angles of the robot (format: [base, joint1, joint2, joint3, gripper])
    # targetTime: Time to reach the target angles (s)
    def setJoints(self, targetAngles, targetTime = None):
        if targetTime is not None:
            # Prepare array of angles to reach the target angles
            # Apply easing function to the angles
            # Calculate the delay for each step
            initAngles = np.array([self.servos[i].angle for i in range(5)])
            angles = np.array([init + self.ease * (target - init) for init, target in zip(initAngles, targetAngles)])
            delay = targetTime / 256
            
            # Send servo commands to the robot step by step
            for step in range(256):
                params = " ".join(np.array([self.servos[i].setAngle(angles[i][step]) for i in range(5)]))
                self.serial.write(f"R 01234 {params}".encode())
                time.sleep(delay)
        else:
            # Send servo commands to the robot
            # Update the servo angles simultaneously
            params = " ".join(np.array([self.servos[i].setAngle(targetAngles[i]) for i in range(5)]))
            self.serial.write(f"R 01234 {params}".encode())
        
    # Set the base of the robot
    # targetAngle: Target angle of the base rotation
    # 0 (left) -> 180 (right)
    # targetTime: Time to reach the target angle (s)
    def setBase(self, targetAngle, targetTime = None):
        if targetTime is not None:
            # Apply easing function to the angle
            # Calculate the delay for each step
            angle = self.servos[1].angle + self.ease * (targetAngle - self.servos[1].angle)
            delay = targetTime / 256
            
            # Send servo commands to the robot step by step
            for step in range(256):
                params = self.servos[1].setAngle(angle[step])
                self.serial.write(f"R 1 {params}".encode())
                time.sleep(delay)
        else:
            # Just update the base angle
            params = self.servos[1].setAngle(targetAngle)
            self.serial.write(f"R 1 {params}".encode())
        
    # Set the gripper of the robot
    # targetAngle: Target angle of the gripper
    # 0 (close) -> 60 (open)
    # Not recommended to set > 60 (too wide)
    # targetTime: Time to reach the target angle (s)
    def setGripper(self, targetAngle, targetTime = None):
        if targetTime is not None:
            # Apply easing function to the angle
            # Calculate the delay for each step
            angle = self.servos[0].angle + self.ease * (targetAngle - self.servos[0].angle)
            delay = targetTime / 256
            
            # Send servo commands to the robot step by step
            for step in range(256):
                params = self.servos[0].setAngle(angle[step])
                self.serial.write(f"R 0 {params}".encode())
                time.sleep(delay)
        else:
            # Just update the gripper angle
            params = self.servos[0].setAngle(targetAngle)
            self.serial.write(f"R 0 {params}".encode())

    # Set the stance of the robot
    # Use inverse kinematics to calculate the joint angles
    # rot: Rotation of the base [left-right] (degree)
    # depth: Depth of the robot [forward-backward] (mm)
    # height: Height of the robot [up-down] (mm)
    # grip: Angle of the gripper [close-open] (degree)
    # targetTime: Time to reach the target stance (s)
    def setStance(self, rot, depth, height, grip, targetTime = None):
        # Update the stance
        self.stance = np.array([depth, height])
        # Calculate the inverse kinematics
        ik = self.model.inverse_kinematics(np.array([0.0e-3, depth*1e-3, height*1e-3]), optimizer='scalar')

        ik[3] = np.pi - ik[3] # Invert the joint 3 angle
        ik = np.degrees(ik) # Convert the angles to degrees
        ik[0] = grip # Set the gripper angle
        ik[1] = rot # Set the base rotation angle
        ik[4] = ik[3] - ik[2] # Calculate the 3rd link angle (always parallel to the ground)

        # Set the joint angles
        self.setJoints(ik, targetTime)
        
    # Robot predefined action
    def action(self):
        # Infinite loop to run the predefined action
        while True:
            # Suppose be able to stack 5 objects
            for i in range(5):
                self.setStance(0, Stance.SCN[0], Stance.SCN[1], 60) # Scanning stance
                _, dis = self.scanObject(self.distTh) # Scan the object
                _, dis = self.scanElevation(self.distTh) # Scan the elevation
                _, dis = self.grabObject(dis) # Grab the object
                _, dis = self.liftObject(dis, self.liftTh) # Lift the object
                _, dis = self.moveObject(dis, self.moveTgt) # Move the object
                _, dis = self.liftObject(dis, 50*i - self.liftTh) # Drop the object
                _, dis = self.grabObject(-dis) # Release the object
        
    # Scan the object
    # threshold: Distance threshold for the robot to detect an object (mm)
    def scanObject(self, threshold=100):
        objStart = None
        lastRotation = 0
        scanUp = True
        while True:
            for rotation in range(0,self.scnArea, 1) if scanUp else range(self.scnArea, 0, -1):
                self.setBase(rotation)
                dist = self.readAndParse('DIST')

                if  dist <= threshold and objStart is None:
                    objStart = rotation
                elif dist > threshold and objStart is not None:
                    objStop = lastRotation
                    rotation = objStart + (objStop - objStart) / 2
                    self.setBase(rotation)
                    dist = self.readAndParse('DIST')
                    return True, dist
                lastRotation = rotation
            scanUp = not scanUp
            
    # Scan the object
    # This is experimental function
    # threshold: Distance threshold for the robot to detect an object (mm)
    # scanInterval: Rotation interval for the robot move (CCR unit)
    def EXPscanObject(self, threshold=100, scanInterval=5):
        self.serial.write(f"F 0 1 {threshold} 0 {self.scnArea} {scanInterval}".encode())
        dist, self.servos[1].raw, self.servos[1].angle = self.readAndParse('DIST RAW1 ROT1')
        return True, dist
        
    # Scan the elevation
    # threshold: Distance threshold for the robot to detect an object (mm)
    def scanElevation(self, threshold=100):
        for elev in range(self.stance[1], self.liftTh, 10):
            self.setStance(self.servos[1].angle, self.stance[0], elev, 60)
            dist = self.readAndParse('DIST')
            if dist > threshold:
                self.setStance(self.servos[1].angle, self.stance[0], elev - 35, 60)
                dist = self.readAndParse('DIST')
                return True, dist
        dist = self.readAndParse('DIST')
        return False, dist

    # Grab the object
    # distance: Distance to check grabbed object (mm)
    def grabObject(self, distance, grabTime = None, gripTime = None):
        self.setStance(self.servos[1].angle, self.stance[0] + distance, self.stance[1], 60, grabTime)
        dist = self.readAndParse('DIST')
        if dist <= distance:
            self.setGripper(0, gripTime)
            return True, dist
        else:
            return False, dist

    # Lift the object
    # distance: Distance to check lifted object (mm)
    # lift: Lift value (mm)
    def liftObject(self, distance, lift, liftTime = None):
        self.setStance(self.servos[1].angle, self.stance[0], self.stance[1] + lift, 0, liftTime)
        dist = self.readAndParse('DIST')
        if dist <= distance:
            return True, dist
        else:
            return False, dist

    # Move the object
    # distance: Distance to check moved object
    # move: Move value (degree)
    def moveObject(self, distance, move, moveTime = None):
        self.setBase(move, moveTime)
        dist = self.readAndParse('DIST')
        if dist <= distance:
            return True, dist
        else:
            return False, dist

In [12]:
# Create the robot object
robot = ArmRobot()

In [15]:
robot.setStance(90, 250, 70, 45)

In [ ]:
# Value Dictionary
# Use this set as standard stance reference
# [174.5, 25.0]
# [174.5, 260.0]

# [185.2, 25.0]
# [185.2, 50.0]
# [185.2, 260.0]

# [191.6, 25.0]
# [191.6, 50.0]
# [191.6, 70.0]
# [191.6, 260.0]

# [242.5, 25.0]
# [242.5, 50.0]
# [242.5, 70.0]
# [242.5, 260.0]